In [ ]:
import folium
import pandas as pd
from folium.features import DivIcon

In [ ]:
data = pd.read_csv('assigned_data.csv')[['centroid_lat','centroid_lon']]
df = data.dropna()
df

,centroid_lat,centroid_lon
0,42.333224,-83.040325
1,42.334716,-83.055451
2,42.325359,-83.054263
3,42.335829,-83.033044
4,42.368950,-83.088561
5,42.397022,-82.970768
6,42.411061,-83.134192
7,42.404382,-83.058488
8,42.327171,-83.126427
9,42.412944,-82.920713


In [ ]:
pairing =  pd.read_csv('paired_result.csv')
pairing = pairing[['Station 1','Station 2']]
connections = list(pairing.itertuples(index=False, name=None))

In [ ]:
connections = [(18, 20), (4, 6), (12, 13), (21, 22), (23, 25), (2, 8), (1, 12), (13, 14), (7, 10),
 (15, 17), (3, 9), (5, 9), (14, 24), (19, 23), (11, 25), (6, 10), (15, 19), (16, 18),
 (20, 21), (5, 8), (22, 24), (1, 4), (2, 3), (7, 11), (16, 17)]


In [ ]:
key_location = {
    "Location": [
        "Airport",
        # "The Fillmore Detroit",
        "The University of Detroit Mercy",
        "Henry Ford Museum of American Innovation",
        "Detroit Historical Museum",
        "Ford Field",
    ],
    # "centroid_lat": [42.2124, 42.3368, 42.4147, 42.3009, 42.36, 42.339],
    # "centroid_lon": [-83.3534, -83.0504, -83.1386, -83.2331, -83.0652, -83.0456],
     "centroid_lat": [42.2124, 42.4147, 42.3009, 42.36, 42.339],
    "centroid_lon": [-83.3534, -83.1386, -83.2331, -83.0652, -83.0456],
}
key_location = pd.DataFrame(key_location)


In [ ]:
# Create a map centered on the United States
map_center = [42.302516, -83.34390]  # Approximate center of the US
us_map = folium.Map(location=map_center, zoom_start=10)

# Add numbered circle markers to the map
for i, row in df.iterrows():
    # Add the circle marker
    folium.CircleMarker(
        location=[row['centroid_lat'], row['centroid_lon']],
        radius=8,  # Circle size
        color='blue',  # Circle border color
        fill=True,
        fill_color='green',  # Fill color
        fill_opacity=0.7
    ).add_to(us_map)

    # Add the number as a label
    folium.map.Marker(
        [row['centroid_lat']-0.005, row['centroid_lon']+0.005],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{i+1}</div>'
        )
    ).add_to(us_map)

# Add stars and labels for key locations
for i, row in key_location.iterrows():
    # Add the star marker
    folium.Marker(
        location=[row["centroid_lat"], row["centroid_lon"]],
        icon=folium.Icon(icon="star", color="red"),
    ).add_to(us_map)

    # Add the label as text
    folium.map.Marker(
        [row["centroid_lat"]+0.005, row["centroid_lon"]+0.015],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{row["Location"]}</div>'
        )
    ).add_to(us_map)

us_map


In [ ]:
original_data = pd.read_csv('aggregated_data.csv')
original_data

,Census_Tract,Num_lived,Num_worked,centroid_lat,centroid_lon
0,5004,198,56,42.426156,-82.979339
1,5005,440,42,42.424134,-82.967943
2,5006,767,53,42.431479,-82.965317
3,5007,1139,76,42.429614,-82.947336
4,5008,875,8,42.424657,-82.935196
...,...,...,...,...,...
319,5755,1728,277,42.285043,-83.223147
320,5756,801,599,42.291689,-83.213875
321,5785,2016,746,42.287881,-83.178865
322,9852,46,7262,42.378578,-82.971522


In [ ]:
import pandas as pd
import folium
from geopy.distance import geodesic

def find_closest_station(row, stations):
    original_point = (row['centroid_lat'], row['centroid_lon'])
    stations['distance'] = stations.apply(
        lambda x: geodesic(original_point, (x['centroid_lat'], x['centroid_lon'])).meters, axis=1
    )
    closest_station = stations.loc[stations['distance'].idxmin()]
    return closest_station['centroid_lat'], closest_station['centroid_lon']

original_data[['closest_station_lat', 'closest_station_lon']] = original_data.apply(
    lambda row: pd.Series(find_closest_station(row, df)), axis=1
)

# Visualize on a Folium map
m = folium.Map(location=[original_data['centroid_lat'].mean(), original_data['centroid_lon'].mean()], zoom_start=11)

# Add all 324 locations
for _, row in original_data.iterrows():
    folium.CircleMarker(
        location=(row['centroid_lat'], row['centroid_lon']),
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6,
    ).add_to(m)

# Add the 25 selected stations
for _, row in df.iterrows():
    folium.Marker(
        location=(row['centroid_lat'], row['centroid_lon']),
        icon=folium.Icon(color='red', icon='train', prefix='fa'),
    ).add_to(m)

# Draw lines from each 324-location to its closest station
for _, row in original_data.iterrows():
    folium.PolyLine(
        locations=[
            (row['centroid_lat'], row['centroid_lon']),
            (row['closest_station_lat'], row['closest_station_lon'])
        ],
        color='green',
        weight=2,
        opacity=0.5,
    ).add_to(m)

m


In [ ]:
import folium
import pandas as pd
from folium.features import DivIcon


key_location = {
    "Location": [
        "Airport",
        "The Fillmore Detroit",
        "The University of Detroit Mercy",
        "Henry Ford Museum of American Innovation",
        "Detroit Historical Museum",
        "Ford Field",
    ],
    "centroid_lat": [42.2124, 42.3368, 42.4147, 42.3009, 42.36, 42.339],
    "centroid_lon": [-83.3534, -83.0504, -83.1386, -83.2331, -83.0652, -83.0456],
}
key_location = pd.DataFrame(key_location)




# Create a map centered on the United States
map_center = [42.302516, -83.34390]  # Approximate center of the region
us_map = folium.Map(location=map_center, zoom_start=10)



for i, row in df.iterrows():
    # Add the circle marker
    folium.CircleMarker(
        location=[row['centroid_lat'], row['centroid_lon']],
        radius=8,  # Circle size
        color='blue',  # Circle border color
        fill=True,
        fill_color='green',  # Fill color
        fill_opacity=0.7
    ).add_to(us_map)

    # Add the number as a label
    folium.map.Marker(
        [row['centroid_lat']-0.005, row['centroid_lon']+0.005],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{i+1}</div>'
        )
    ).add_to(us_map)


# Draw lines between connected stations
for start, end in connections:
    start = start -1
    end = end -1
    coords = [
        [df.loc[start, 'centroid_lat'], df.loc[start, 'centroid_lon']],
        [df.loc[end, 'centroid_lat'], df.loc[end, 'centroid_lon']]
    ]
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=2  # Line thickness
    ).add_to(us_map)


# Add stars for key locations
for i, row in key_location.iterrows():
    folium.Marker(
        location=[row["centroid_lat"], row["centroid_lon"]],
        icon=folium.Icon(icon="star", color="red"),
        popup=row["Location"],
    ).add_to(us_map)


# Display the map
us_map


In [ ]:
import folium
import pandas as pd
from folium.features import DivIcon

# Create a map centered on the United States
map_center = [42.302516, -83.34390]  # Approximate center of the region
us_map = folium.Map(location=map_center, zoom_start=11)

# Add numbered circle markers to the map
for i, row in df.iterrows():
    # Add the circle marker
    folium.CircleMarker(
        location=[row['centroid_lat'], row['centroid_lon']],
        radius=8,  # Circle size
        color='blue',  # Circle border color
        fill=True,
        fill_color='green',  # Fill color
        fill_opacity=0.7
    ).add_to(us_map)

    # Add the number as a label
    folium.map.Marker(
        [row['centroid_lat'], row['centroid_lon']],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{i+1}</div>'
        )
    ).add_to(us_map)

# Draw lines between connected stations
for start, end in connections:
    start = start -1
    end = end -1
    coords = [
        [df.loc[start, 'centroid_lat'], df.loc[start, 'centroid_lon']],
        [df.loc[end, 'centroid_lat'], df.loc[end, 'centroid_lon']]
    ]
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=2  # Line thickness
    ).add_to(us_map)

# Display the map
us_map


In [ ]:
import folium
import pandas as pd
from folium.features import DivIcon

# Create a map centered on the United States
map_center = [42.302516, -83.34390]  # Approximate center of the region
us_map = folium.Map(location=map_center, zoom_start=11)

# List of stations to mark as yellow
yellow_stations = [16, 20, 2, 15]

# Add numbered circle markers to the map
for i, row in df.iterrows():
    # Determine the fill color based on station number
    fill_color = 'red' if (i + 1) in yellow_stations else 'green'

    # Add the circle marker
    folium.CircleMarker(
        location=[row['centroid_lat'], row['centroid_lon']],
        radius=8,  # Circle size
        color='blue',  # Circle border color
        fill=True,
        fill_color=fill_color,  # Fill color
        fill_opacity=0.7
    ).add_to(us_map)

    # Add the number as a label
    folium.map.Marker(
        [row['centroid_lat']-0.005, row['centroid_lon']+0.005],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: black; font-weight: bold;">{i+1}</div>'
        )
    ).add_to(us_map)

# Remove the line between Station 7 and Station 10 by skipping it in the connections loop
for start, end in connections:
    # Skip the connection between Station 7 and Station 10
    if (start == 7 and end == 10) or (start == 10 and end == 7):
        continue

    start = start - 1
    end = end - 1
    coords = [
        [df.loc[start, 'centroid_lat'], df.loc[start, 'centroid_lon']],
        [df.loc[end, 'centroid_lat'], df.loc[end, 'centroid_lon']]
    ]
    folium.PolyLine(
        locations=coords,
        color='blue',
        weight=2  # Line thickness
    ).add_to(us_map)

# Add new connections: Station 7 to Station 5 and Station 5 to Station 10
station_7_coords = [df.loc[6, 'centroid_lat'], df.loc[6, 'centroid_lon']]  # Index 6 for Station 7
station_5_coords = [df.loc[7, 'centroid_lat'], df.loc[7, 'centroid_lon']]  # Index 4 for Station 5
station_10_coords = [df.loc[9, 'centroid_lat'], df.loc[9, 'centroid_lon']]  # Index 9 for Station 10

# Line from Station 7 to Station 5
folium.PolyLine(
    locations=[station_7_coords, station_5_coords],
    color='blue',
    weight=2
).add_to(us_map)

# Line from Station 5 to Station 10
folium.PolyLine(
    locations=[station_5_coords, station_10_coords],
    color='blue',
    weight=2
).add_to(us_map)

# Add a line from Station 15 to given coordinates (42.2124, -83.3534)
station_15_coords = [df.loc[14, 'centroid_lat'], df.loc[14, 'centroid_lon']]  # Index 14 for Station 15
line_to_coords = [42.2124, -83.3534]
folium.PolyLine(
    locations=[station_15_coords, line_to_coords],
    color='blue',
    weight=2
).add_to(us_map)

# Add a line connecting Station 15 and Station 12
station_12_coords = [df.loc[11, 'centroid_lat'], df.loc[11, 'centroid_lon']]  # Index 11 for Station 12
folium.PolyLine(
    locations=[station_15_coords, station_12_coords],
    color='blue',
    weight=2
).add_to(us_map)

# Add a circle with an airport logo at (42.2124, -83.3534)
folium.Marker(
    location=[42.2124, -83.3534],
    icon=folium.Icon(icon='plane', prefix='fa', color='blue')
).add_to(us_map)

# Display the map
us_map
